In [1]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)

ValueError: Length of values (81300) does not match length of index (40650)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Define your models and their hyperparameters
models = [
    ('Logistic Regression', LogisticRegression(), {'C': [0.1, 1, 10]}),
    ('Random Forest', RandomForestClassifier(), {'n_estimators': [10, 50, 100], 'max_depth': [None, 10, 20]}),
    ('SVM', SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
    ('Neural Network', MLPClassifier(), {'hidden_layer_sizes': [(10,), (50,), (100,)], 'activation': ['relu', 'tanh']})
]

# Use cross-validation and grid search to compare the models
for name, model, params in models:
    grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    print(f"{name} Best Parameters: {grid_search.best_params_}")
    print(f"{name} Best Score: {grid_search.best_score_}")
